In [11]:
# Create Lat and Long Combos
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime

In [12]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

#This zip objects packs each pair of lats and lngs having the same index in their respective array into a tuple


In [13]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [14]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [15]:
!pip install gmaps

In [16]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

608

# 2.6 Retreive City Weather Data 

In [20]:
# Import the API key.
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [28]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    #Code Breakdown
    #1. create a for loop with enumerate() method and reference the index and the city in the list
    #2. Check if remainder of index divide by 50 equals 0 and if index is greater than or equal to 50. If statement true
    # then set_count and record_count are incremented by 1
    #3. inside coniditonal statement we create URL endpoint for each city as before, we are removing blank spaces in city name
    # and concatenating city name with city.replace(" ","+") which will find corresponding weather data for city instead of data
    #for first part of city name
    #4. add print statement that tells us record count, set count and city being processed
    #5. add one to the record count before next city is processed
 

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hasaki
Processing Record 2 of Set 1 | tautira
Processing Record 3 of Set 1 | lobito
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | tasiilaq
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | atuona
Processing Record 8 of Set 1 | agadir
Processing Record 9 of Set 1 | champasak
Processing Record 10 of Set 1 | nicoya
Processing Record 11 of Set 1 | georgetown
Processing Record 12 of Set 1 | kapaa
Processing Record 13 of Set 1 | preobrazhenskaya
City not found. Skipping...
Processing Record 14 of Set 1 | the valley
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | vaini
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | nishihara
Processing Record 19 of Set 1 | yellowknife
Processing Record 20 of Set 1 | lulea
Processing Record 21 of Set 1 | bahia blanca
Processing Record 22 of Set 1 | tura
Processing Record 23 of Set

Processing Record 38 of Set 4 | kermanshah
Processing Record 39 of Set 4 | cidreira
Processing Record 40 of Set 4 | norman wells
Processing Record 41 of Set 4 | sorvag
City not found. Skipping...
Processing Record 42 of Set 4 | ojinaga
Processing Record 43 of Set 4 | vestmannaeyjar
Processing Record 44 of Set 4 | aykhal
Processing Record 45 of Set 4 | castro
Processing Record 46 of Set 4 | longyearbyen
Processing Record 47 of Set 4 | gat
Processing Record 48 of Set 4 | tumannyy
City not found. Skipping...
Processing Record 49 of Set 4 | santa maria
Processing Record 50 of Set 4 | hithadhoo
Processing Record 1 of Set 5 | coihaique
Processing Record 2 of Set 5 | pochutla
Processing Record 3 of Set 5 | te anau
Processing Record 4 of Set 5 | cherskiy
Processing Record 5 of Set 5 | kaitangata
Processing Record 6 of Set 5 | padang
Processing Record 7 of Set 5 | fort nelson
Processing Record 8 of Set 5 | pidhorodne
Processing Record 9 of Set 5 | kolyvan
Processing Record 10 of Set 5 | nguiu
C

Processing Record 26 of Set 8 | yaan
Processing Record 27 of Set 8 | siocon
Processing Record 28 of Set 8 | mansa
Processing Record 29 of Set 8 | axim
Processing Record 30 of Set 8 | goderich
Processing Record 31 of Set 8 | jumla
Processing Record 32 of Set 8 | sakaraha
Processing Record 33 of Set 8 | mahebourg
Processing Record 34 of Set 8 | saleaula
City not found. Skipping...
Processing Record 35 of Set 8 | rawson
Processing Record 36 of Set 8 | mnogovershinnyy
Processing Record 37 of Set 8 | bogashevo
Processing Record 38 of Set 8 | aflu
City not found. Skipping...
Processing Record 39 of Set 8 | lorengau
Processing Record 40 of Set 8 | samarai
Processing Record 41 of Set 8 | cabinda
Processing Record 42 of Set 8 | raubling
Processing Record 43 of Set 8 | bethel
Processing Record 44 of Set 8 | candolim
Processing Record 45 of Set 8 | mitsamiouli
Processing Record 46 of Set 8 | karamay
City not found. Skipping...
Processing Record 47 of Set 8 | satitoa
City not found. Skipping...
Pr

Processing Record 12 of Set 12 | ambon
Processing Record 13 of Set 12 | teya
Processing Record 14 of Set 12 | neuquen
Processing Record 15 of Set 12 | alice springs
Processing Record 16 of Set 12 | hay river
Processing Record 17 of Set 12 | seymchan
Processing Record 18 of Set 12 | zhezkazgan
Processing Record 19 of Set 12 | sentyabrskiy
City not found. Skipping...
Processing Record 20 of Set 12 | pisco
Processing Record 21 of Set 12 | brownfield
Processing Record 22 of Set 12 | camacha
Processing Record 23 of Set 12 | paradwip
City not found. Skipping...
Processing Record 24 of Set 12 | rupnagar
Processing Record 25 of Set 12 | port-gentil
Processing Record 26 of Set 12 | dhidhdhoo
Processing Record 27 of Set 12 | niamey
Processing Record 28 of Set 12 | ninghai
Processing Record 29 of Set 12 | varkkallai
Processing Record 30 of Set 12 | ust-kamchatsk
City not found. Skipping...
Processing Record 31 of Set 12 | griffith
Processing Record 32 of Set 12 | huarmey
Processing Record 33 of S

In [31]:
#2.7 Create a Data Frame
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hasaki,35.7333,140.8333,60.80,51,20,2.30,JP,2021-04-19 23:30:29
1,Tautira,-17.7333,-149.1500,82.40,78,75,6.91,PF,2021-04-19 23:30:30
2,Lobito,-12.3481,13.5456,80.28,80,64,2.28,AO,2021-04-19 23:30:30
3,Albany,42.6001,-73.9662,57.99,56,100,3.80,US,2021-04-19 23:26:55
4,Tasiilaq,65.6145,-37.6368,24.80,68,15,11.50,GL,2021-04-19 23:30:30
5,Ushuaia,-54.8000,-68.3000,42.80,87,40,21.85,AR,2021-04-19 23:30:31
6,Atuona,-9.8000,-139.0333,80.51,76,23,18.57,PF,2021-04-19 23:30:31
7,Agadir,30.4202,-9.5982,60.80,82,0,5.75,MA,2021-04-19 23:28:46
8,Champasak,14.7500,106.0000,73.76,93,100,2.10,LA,2021-04-19 23:30:32
9,Nicoya,10.1483,-85.4520,71.60,94,75,10.36,CR,2021-04-19 23:30:32


In [32]:
# Set new column order
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
# Set dataframe to accept new column order
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Hasaki,JP,2021-04-19 23:30:29,35.7333,140.8333,60.80,51,20,2.30
1,Tautira,PF,2021-04-19 23:30:30,-17.7333,-149.1500,82.40,78,75,6.91
2,Lobito,AO,2021-04-19 23:30:30,-12.3481,13.5456,80.28,80,64,2.28
3,Albany,US,2021-04-19 23:26:55,42.6001,-73.9662,57.99,56,100,3.80
4,Tasiilaq,GL,2021-04-19 23:30:30,65.6145,-37.6368,24.80,68,15,11.50
5,Ushuaia,AR,2021-04-19 23:30:31,-54.8000,-68.3000,42.80,87,40,21.85
6,Atuona,PF,2021-04-19 23:30:31,-9.8000,-139.0333,80.51,76,23,18.57
7,Agadir,MA,2021-04-19 23:28:46,30.4202,-9.5982,60.80,82,0,5.75
8,Champasak,LA,2021-04-19 23:30:32,14.7500,106.0000,73.76,93,100,2.10
9,Nicoya,CR,2021-04-19 23:30:32,10.1483,-85.4520,71.60,94,75,10.36


In [33]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")